# Transforming registrant datasets
## Author: Karina Lopez
### Last updated: 07/22/2021

# Things that need to be revised:
- clean up registration numbers that are grouped and mistakenly input as different rows (e.g., 'HSR19-003524')
- Leave nulls alone (e.g., HSR20-002175)
- two addresses to same place: (e.g., HSR19-004463); two address entries for the same registrant ID number); maybe explain it if its the only one; if there are multiple ones, can explain multiple addresses can exist for one registrant number
- remove mailing address
- next week we can give Jon an update on progress

In [184]:
import pandas as pd
import glob
import os
import numpy as np
from collections import OrderedDict

BASE_DIR = "/Users/karinalopez/Desktop/ds_projects/hack4la_airbnb/data/"

# Functions

In [185]:
def merge_dfs(df1, df2, col1, col2, fcol, first_bool):
    
    # merge each dataset together
    df = pd.merge(df1, df2, how = 'outer', 
                         on = ['registration_num', 'address1', 'address2', 'registrant_name'],
                         suffixes = ['1', '2'])
    
    return(merge_n_drop(df, col1, col2, fcol, first_bool))

In [186]:
def merge_n_drop(df, col1, col2, fcol, first_bool):
    
    # combine platforms listed
    cols = [col1, col2]
    df[fcol] = df[cols].apply(lambda x: ' '.join(x.dropna()), axis = 1)

    # remove duplicate platforms
    df[fcol] = (df[fcol].str.split()
                                  .apply(lambda x: OrderedDict.fromkeys(x).keys())
                                  .str.join(','))
    
    if first_bool: # only when you want to drop the first two original columns
        # drop unnecessary columns
        df = df.drop([col1, col2], axis = 1)
    
    else:
        # only drop the individual date column
        df = df.drop([col2], axis = 1)

    return(df)

# Merge city of LA registerant datasets

In [187]:
os.chdir(BASE_DIR + 'raw/City of LA data/')
xls = pd.ExcelFile('HSO Registrants over time.xlsx')


In [188]:
cityla_df1 = pd.read_excel(xls, '12-12-19')
cityla_df2 = pd.read_excel(xls, '1-16-20')
cityla_df3 = pd.read_excel(xls, '2-7-20')
cityla_df4 = pd.read_excel(xls, '3-13-20')
cityla_df5 = pd.read_excel(xls, '4-14-20')
cityla_df6 = pd.read_excel(xls, '5-27-20')
cityla_df7 = pd.read_excel(xls, '7-9-20')
cityla_df8 = pd.read_excel(xls, '12-4-20')
cityla_df9 = pd.read_excel(xls, '12-21-20')
cityla_df10 = pd.read_excel(xls, '2-17-21')
cityla_df11 = pd.read_excel(xls, '4-28-21')
cityla_df12 = pd.read_excel(xls, '5-18-21 extended')


# Checkpoint 1: Rename columns

In [189]:
# create dictionary with all columns to be renamed
col_dict = {'Registration Number': 'registration_num', 
     'Registration\nNumber': 'registration_num',
     'Date Generated': 'registration_date',
     'Registration Date': 'registration_date', 
     'Registered Unit\nNumber': 'address2',
     'Registered Unit Number': 'address2', 
     'Property Unit Number': 'address2',
     'Registered Adddress': 'address1',
     'Registered Address': 'address1',
     'Property Address': 'address1', 
     'Permit Holder Name 1': 'registrant_name',
     'Permit Holder Name 1 ': 'registrant_name',
     'Permit Holder Name': 'registrant_name', 
     'Permit Holder': 'registrant_name', 
     'Name of Host/Registrant': 'registrant_name', 
     'Registrant Name': 'registrant_name',
     'Platforms Reported at Application': 'platforms_listed',
     'Listings': 'platforms_listed',
     'Registered Platform(s)': 'platforms_listed',
     'Platforms': 'platforms_listed',
     'Platform': 'platforms_listed'}


In [190]:
# rename columns for each dataframe if the column name is listed in the dictionary
cityla_df1.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df1}, inplace = True)
cityla_df2.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df2}, inplace = True)
cityla_df3.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df3}, inplace = True)
cityla_df4.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df4}, inplace = True)
cityla_df5.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df5}, inplace = True)
cityla_df6.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df6}, inplace = True)
cityla_df7.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df7}, inplace = True)
cityla_df8.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df8}, inplace = True)
cityla_df9.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df9}, inplace = True)
cityla_df10.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df10}, inplace = True)
cityla_df11.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df11}, inplace = True)
cityla_df12.rename(columns = {k: v for k, v in col_dict.items() if v not in cityla_df12}, inplace = True)


In [191]:
# change dates to proper format: YYYY-MM-DD
cityla_df9['registration_date'] = pd.to_datetime(cityla_df9['registration_date'], format = '%y-%m-%d %h:%m:%s').dt.strftime('%Y-%m-%d')

In [192]:
cityla_df9.isna().any()

registration_num     False
address1             False
address2              True
registrant_name      False
platforms_listed      True
registration_date    False
dtype: bool

In [193]:
# convert everything to uppercase letters
cityla_df1 = cityla_df1.apply(lambda x: x.astype(str).str.upper()) 
cityla_df2 = cityla_df2.apply(lambda x: x.astype(str).str.upper())
cityla_df3 = cityla_df3.apply(lambda x: x.astype(str).str.upper())
cityla_df4 = cityla_df4.apply(lambda x: x.astype(str).str.upper())
cityla_df5 = cityla_df5.apply(lambda x: x.astype(str).str.upper())
cityla_df6 = cityla_df6.apply(lambda x: x.astype(str).str.upper())
cityla_df7 = cityla_df7.apply(lambda x: x.astype(str).str.upper())
cityla_df8 = cityla_df8.apply(lambda x: x.astype(str).str.upper())
cityla_df9 = cityla_df9.apply(lambda x: x.astype(str).str.upper())
cityla_df10 = cityla_df10.apply(lambda x: x.astype(str).str.upper())
cityla_df11 = cityla_df11.apply(lambda x: x.astype(str).str.upper())
cityla_df12 = cityla_df12.apply(lambda x: x.astype(str).str.upper())

In [194]:
cityla_df1 = cityla_df1.replace('-', np.nan)
cityla_df2 = cityla_df2.replace('-', np.nan)
cityla_df3 = cityla_df3.replace('-', np.nan)
cityla_df4 = cityla_df4.replace('-', np.nan)
cityla_df5 = cityla_df5.replace('-', np.nan)
cityla_df6 = cityla_df6.replace('-', np.nan)
cityla_df7 = cityla_df7.replace('-', np.nan)
cityla_df8 = cityla_df8.replace('-', np.nan)
cityla_df9 = cityla_df9.replace('-', np.nan)
cityla_df10 = cityla_df10.replace('-', np.nan)
cityla_df11 = cityla_df11.replace('-', np.nan)
cityla_df12 = cityla_df12.replace('-', np.nan)

cityla_df1['address2'] = cityla_df1['address2'].replace('NAN', np.nan)
cityla_df2['address2'] = cityla_df2['address2'].replace('NAN', np.nan)
cityla_df3['address2'] = cityla_df3['address2'].replace('NAN', np.nan)
cityla_df4['address2'] = cityla_df4['address2'].replace('NAN', np.nan)
cityla_df5['address2'] = cityla_df5['address2'].replace('NAN', np.nan)
cityla_df6['address2'] = cityla_df6['address2'].replace('NAN', np.nan)
cityla_df7['address2'] = cityla_df7['address2'].replace('NAN', np.nan)
cityla_df8['address2'] = cityla_df8['address2'].replace('NAN', np.nan)
cityla_df9['address2'] = cityla_df9['address2'].replace('NAN', np.nan)
cityla_df10['address2'] = cityla_df10['address2'].replace('NAN', np.nan)
cityla_df11['address2'] = cityla_df11['address2'].replace('NAN', np.nan)
cityla_df12['address2'] = cityla_df12['address2'].replace('NAN', np.nan)

cityla_df1['platforms_listed'] = cityla_df1['platforms_listed'].replace('NAN', np.nan)
cityla_df2['platforms_listed'] = cityla_df2['platforms_listed'].replace('NAN', np.nan)
cityla_df3['platforms_listed'] = cityla_df3['platforms_listed'].replace('NAN', np.nan)
cityla_df4['platforms_listed'] = cityla_df4['platforms_listed'].replace('NAN', np.nan)
cityla_df5['platforms_listed'] = cityla_df5['platforms_listed'].replace('NAN', np.nan)
cityla_df6['platforms_listed'] = cityla_df6['platforms_listed'].replace('NAN', np.nan)
cityla_df7['platforms_listed'] = cityla_df7['platforms_listed'].replace('NAN', np.nan)
cityla_df8['platforms_listed'] = cityla_df8['platforms_listed'].replace('NAN', np.nan)
cityla_df9['platforms_listed'] = cityla_df9['platforms_listed'].replace('NAN', np.nan)
cityla_df10['platforms_listed'] = cityla_df10['platforms_listed'].replace('NAN', np.nan)
cityla_df11['platforms_listed'] = cityla_df11['platforms_listed'].replace('NAN', np.nan)
cityla_df12['platforms_listed'] = cityla_df12['platforms_listed'].replace('NAN', np.nan)

In [195]:
# create new column containing date of registration for each dataset
date1 = '12-12-19'
date2 = '1-16-20'
date3 = '2-7-20'
date4 = '3-13-20'
date5 = '4-14-20'
date6 = '5-27-20'
date7 = '7-9-20'
date8 = '12-4-20'
date9 = '12-21-20'
date10 = '2-17-21'
date11 = '4-28-21'
date12 = '5-18-21'

cityla_df1['date1'] = date1
cityla_df2['date2'] = date2
cityla_df3['date3'] = date3
cityla_df4['date4'] = date4
cityla_df5['date5'] = date5
cityla_df6['date6'] = date6
cityla_df7['date7'] = date7
cityla_df8['date8'] = date8
cityla_df9['date9'] = date9
cityla_df10['date10'] = date10
cityla_df11['date11'] = date11
cityla_df12['date12'] = date12


In [256]:
# df 1 + 2
cityla_df = merge_dfs(cityla_df1, cityla_df2, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date1', 'date2', 'date', True)

# df 3
cityla_df = merge_dfs(cityla_df, cityla_df3, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date3', 'date', False) 

# df 4
cityla_df = merge_dfs(cityla_df, cityla_df4, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date4', 'date', False) 

# df 5
cityla_df = merge_dfs(cityla_df, cityla_df5, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date5', 'date', False) 

# df 6
cityla_df = merge_dfs(cityla_df, cityla_df6, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date6', 'date', False) 

# df 7
cityla_df = merge_dfs(cityla_df, cityla_df7, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date7', 'date', False) 

# df 8
cityla_df = merge_dfs(cityla_df, cityla_df8, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date8', 'date', False) 

# df 9
cityla_df = merge_dfs(cityla_df, cityla_df9, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date9', 'date', False) 

# df 10
cityla_df = merge_dfs(cityla_df, cityla_df10, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date10', 'date', False) 

# df 11
cityla_df = merge_dfs(cityla_df, cityla_df11, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date11', 'date', False) 

# df 12
cityla_df = merge_dfs(cityla_df, cityla_df12, 'platforms_listed1', 'platforms_listed2', 'platforms_listed', True)
cityla_df = merge_n_drop(cityla_df, 'date', 'date12', 'date', False) 


In [261]:
# remove duplicate platform listings
cityla_df['platforms_listed'] = cityla_df['platforms_listed'].apply(lambda x: list(set(x)))
cityla_df['platforms_listed'] = cityla_df['platforms_listed'].apply(lambda x: [y for y in x if y])

# remove empty lists
cityla_df['platforms_listed'] = cityla_df['platforms_listed'].apply(lambda x: np.nan if len(x) == 0 else x)



In [ ]:
# combine registration dates to single column
cityla_df = merge_n_drop(cityla_df, 'registration_date1', 'registration_date2', 'registration_date', True)



In [276]:
# rename date column # needs to be revised to include when registrants entered/left these datasets
cityla_df.rename(columns = {'date': 'generated_date'}, inplace = True)
cityla_df.head()


,registration_num,address1,address2,registrant_name,generated_date,platforms_listed,registration_date
0,HSR19-000002,"56 OZONE AVE, VENICE, CA 90291, USA",NaN,CHANTEL HUNTERMAH,"12-12-19,1-16-20,2-7-20,3-13-20,4-14-20,5-27-2...","[AIRBNB, VRBO]","2020-09-15,2021-05-18"
1,HSR19-000003,"11822 GOSHEN AVE, LOS ANGELES, CA 90049, USA",102,ELLIOT RUTSTEIN,12-12-19,[AIRBNB],
2,HSR19-000004,"6703 ST CLAIR AVE, NORTH HOLLYWOOD, CA 91606, ...",NaN,RAJESH GARG,12-12-19,[AIRBNB],
3,HSR19-000005,"2192 BEECH KNOLL RD, LOS ANGELES, CA 90046, USA\t",NaN,ERIC ROSS ALLEN\t,12-12-19,[AIRBNB],
4,HSR19-000007,"752 W ATHENS BLVD, LOS ANGELES, CA 90044, USA",NaN,RODOLFO LARA,12-12-19,NaN,


In [270]:
print(cityla_df1.shape)
print(cityla_df2.shape)
print(cityla_df3.shape)
print(cityla_df4.shape)
print(cityla_df5.shape)
print(cityla_df6.shape)
print(cityla_df7.shape)
print(cityla_df8.shape)
print(cityla_df9.shape)
print(cityla_df10.shape)
print(cityla_df11.shape)
print(cityla_df12.shape)
print(cityla_df.shape)

(4684, 6)
(1116, 6)
(1620, 6)
(2434, 6)
(3358, 6)
(3795, 6)
(3977, 6)
(4021, 6)
(4033, 7)
(3439, 6)
(2949, 6)
(820, 7)
(7186, 8)


In [278]:
#os.chdir(BASE_DIR + 'intermediate/')
#cityla_df.to_csv('cityla_checkpoint_1.csv', index = False)

# Checkpoint 2: Clean up addresses and create new columns

In [619]:
os.chdir(BASE_DIR + 'intermediate/')
cityla_df = pd.read_csv('cityla_checkpoint_1.csv')

In [620]:
# remove unnecessary characters
cityla_df['address1'] = cityla_df['address1'].str.replace('\t','')
cityla_df['address1'] = cityla_df['address1'].str.replace('-','')


In [621]:
cityla_df['registrant_name'] = cityla_df['registrant_name'].str.replace('\t','')
cityla_df['registrant_name'] = cityla_df['registrant_name'].str.replace('-','')


In [622]:
# remove unnecessary characters
cityla_df['address1'] = cityla_df['address1'].apply(lambda x: x.split(', usa')[0])


In [623]:
# remove white space at the beginning and end of a string
cityla_df['address1'] = cityla_df['address1'].str.strip()
cityla_df['registrant_name'] = cityla_df['registrant_name'].str.strip()


In [624]:
# fix these addresses manually
address_dict = {
    'los1118 valencia street angeles, ca 90015': '1118 valencia street, los angeles, ca 90015, usa',
    '1506 n mccollum place unit a los angeles, ca 90026': '1506 n mccollum place, los angeles, ca 90026, usa',
    '4710 west maplewood ave los angeles, ca 90004': '4710 west maplewood ave, los angeles, ca 90004, usa',
    '639 s spring st, los angeles, ca 90014, unit 11a': '639 s spring st, los angeles, ca 90014, usa',
    '4832 riverton ave, north hollywood, 91601': '4832 riverton ave, north hollywood, ca 91601, usa',
    
    'los1118 valencia street angeles, ca 90015':   '1118 valencia street, los angeles, ca 90015, usa',
    '1118 valencia street angeles, ca 90015':      '1118 valencia street, los angeles, ca 90015, usa',
    '1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
    '1118 valencia street angeles, ca 90015':      '1118 valencia street, los angeles, ca 90015, usa',
    'los1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
    
    '14333 van nuys blvd.  ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa',
    '2765 1/2 west 12th street':'2765 w 12th St, los angeles, ca 90006, usa',
    '10834 hortense st, #1 north hollywood, ca 91602, usa':'10834 hortense st #1, north hollywood, ca 91602, usa',
                
                'w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91411, usa',
                '13260 w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91423, usa',
                
                '14333 van nuys blvd. ca 91331, usa':'14333 van nuys blvd, arleta, ca 91331, usa',
                '830 e kensington rd los angeles, ca 90026, usa':'830 e kensington rd, los angeles, ca 90026, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa',
                '518 e 21st street los angeles, ca 90011, usa':'518 e 21st st, los angeles, ca 90011, usa',
                '707 brooks ave venice, ca 90291, usa':'707 brooks ave, venice, ca 90291, usa',
                '1255 federal ave, apt 103 los angeles, ca 90025, usa':'1255 federal ave apt 103, los angeles, ca 90025, usa',
                '344 indiana ave, #b venice, ca 90291, usa':'344 indiana ave #b, venice, ca 90291, usa',
                '1506 n mccollum place unit a los angeles, ca 90026, usa':'1506 n mccollum place unit a, los angeles, ca 90026, usa',
                '7641 tujunga ave north hollywood, ca 91605, usa': '7641 tujunga ave, north hollywood, ca 91605, usa',
                '6828 van nuys blvd, n. sherman oaks, ca 91405, usa': '6828 van nuys blvd, sherman oaks, ca 91405, usa', 
                '2763 1/2 w 12th st los angeles, ca 90006, usa': '2763 1/2 w 12th st, los angeles, ca 90006, usa', 
                '1474 ridge way, los angeles b, ca 90026, usa': '1474 ridge way b, los angeles, ca 90026, usa', 
                '6650 leland way, unit c los angeles, ca 90028, usa': '6650 leland way unit c, los angeles, ca 90028, usa', 
                '4712 west maplewood ave los angeles, ca 90004, usa': '4712 west maplewood ave, los angeles, ca 90004, usa', 
                '732 huntley dr, 1 west hollywood, ca 90069, usa': '732 huntley dr 1, west hollywood, ca 90069, usa', 
                '2765 1/2 west 12th street, usa': '2765 1/2 west 12th st, los angeles, ca 90006, usa', 
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa', 
                '4710 west maplewood ave los angeles, ca 90004, usa': '4710 west maplewood ave, los angeles, ca 90004, usa', 
                '14333 van nuys blvd. ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa', 
                '1710 n. fuller ave, ca 90046, usa': '1710 n fuller ave, los angeles, ca 90046, usa', 
                '14618 gilmore st, van nuys, 10, ca 91411, usa': '14618 gilmore st 10, van nuys, ca 91411, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
                '1019 s catalina ct, unit 508 los angeles, ca 90006, usa': '1019 s catalina ct unit 508, los angeles, ca 90006, usa',
                '1118 valencia street angeles, ca 90015': '1118 valencia street, los angeles, ca 90015'}



In [625]:
# convert everything to lowercase temporarily
cityla_df['address1'] = cityla_df['address1'].apply(lambda x: x.lower())


In [626]:
# map revised addresses
cityla_df['address1'] = cityla_df['address1'].replace(address_dict)
cityla_df['address1'] = cityla_df['address1'].apply(lambda x: x.upper())


In [627]:
# remove country info from addresses
remove_address_char = [', UNITED STATES', ', USA', ', US', ' USA']
cityla_df['address1'] = cityla_df['address1'].str.replace('|'.join(remove_address_char), '')


In [628]:
# create city column

# remove last 10 characters
cityla_df['city'] = cityla_df['address1'].map(lambda x: str(x)[:-10])

# split string by delimeter (,)
cityla_df['city'] = cityla_df['city'].str.split(',')

# create city and street address columns
cityla_df.loc[:, 'city'] = cityla_df['city'].map(lambda x: x[-1])

# remove white space 
cityla_df['city'] = cityla_df['city'].str.strip()


In [629]:
cityla_df['city'] = cityla_df['city'].apply(lambda x: x.lower())
cityla_df['address1'] = cityla_df['address1'].apply(lambda x: x.lower())
#cityla_df['city'] = cityla_df['city'].apply(lambda x: x.upper())

In [631]:
# clean up registration numbers that are grouped and mistakenly input as different rows (e.g., 'HSR19-003524')
# Leave nulls alone (e.g., HSR20-002175)
# two addresses to same place: (e.g., HSR19-004463); two address entries for the same registrant ID number); maybe explain it if its the only one; if there are multiple ones, can explain mukltiple addresses can exist for one registrant number
# remove mailing address
# next week we can give Jon an update on progress


cityla_df.loc[cityla_df['city'] == 'san pedr']

# fact table




,registration_num,address1,address2,registrant_name,generated_date,platforms_listed,registration_date,city
3756,HSR19-004463,"2304 w paseo del mar, san pedro, ca 90732 aka ...",NaN,SABRINA MEYN,12-12-19,"['AIRBNB', 'VRBO']",NaN,san pedr


In [588]:
temp = cityla_df['city'].value_counts().rename_axis('unique_values').reset_index(name='counts')
temp.loc[temp['counts'] == 1]


,unique_values,counts
50,1506 n mccollum place unit a los angeles,1
51,1118 valencia street angeles,1
52,san fernando,1
53,#50,1
54,shadow hills (sunland),1
55,simi valley,1
56,san pedr,1
57,unit 508 los angeles,1
58,los1118 valencia street angeles,1
59,4712 west maplewood ave los angeles,1


In [593]:
cityla_df.loc[cityla_df['city'] == '1118 valencia street angeles']


,registration_num,address1,address2,registrant_name,generated_date,platforms_listed,registration_date,city


In [557]:
cityla_df.loc[cityla_df['address1'] == '1118 valencia street angeles, ca 90015']


,registration_num,address1,address2,registrant_name,generated_date,platforms_listed,registration_date,city
5762,HSR19-003524,"1118 valencia street angeles, ca 90015",118,NICHOLAS ORTON,"12-4-20,12-21-20",['AIRBNB'],2020-09-11,1118 valencia street angeles


In [478]:
# fix these addresses manually
address_dict = {
    '1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
    '1118 valencia street angeles, ca 90015': '1118 valencia street, los angeles, ca 90015, usa'
    'los1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
    '14333 van nuys blvd.  ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa',
    '2765 1/2 west 12th street':'2765 w 12th St, los angeles, ca 90006, usa',
    '10834 hortense st, #1 north hollywood, ca 91602, usa':'10834 hortense st #1, north hollywood, ca 91602, usa',
                
                'w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91411, usa',
                '13260 w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91423, usa',
                
                '14333 van nuys blvd. ca 91331, usa':'14333 van nuys blvd, arleta, ca 91331, usa',
                '830 e kensington rd los angeles, ca 90026, usa':'830 e kensington rd, los angeles, ca 90026, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa',
                '518 e 21st street los angeles, ca 90011, usa':'518 e 21st st, los angeles, ca 90011, usa',
                '707 brooks ave venice, ca 90291, usa':'707 brooks ave, venice, ca 90291, usa',
                '1255 federal ave, apt 103 los angeles, ca 90025, usa':'1255 federal ave apt 103, los angeles, ca 90025, usa',
                '344 indiana ave, #b venice, ca 90291, usa':'344 indiana ave #b, venice, ca 90291, usa',
                '1506 n mccollum place unit a los angeles, ca 90026, usa':'1506 n mccollum place unit a, los angeles, ca 90026, usa',
                '7641 tujunga ave north hollywood, ca 91605, usa': '7641 tujunga ave, north hollywood, ca 91605, usa',
                '6828 van nuys blvd, n. sherman oaks, ca 91405, usa': '6828 van nuys blvd, sherman oaks, ca 91405, usa', 
                '2763 1/2 w 12th st los angeles, ca 90006, usa': '2763 1/2 w 12th st, los angeles, ca 90006, usa', 
                '1474 ridge way, los angeles b, ca 90026, usa': '1474 ridge way b, los angeles, ca 90026, usa', 
                '6650 leland way, unit c los angeles, ca 90028, usa': '6650 leland way unit c, los angeles, ca 90028, usa', 
                '4712 west maplewood ave los angeles, ca 90004, usa': '4712 west maplewood ave, los angeles, ca 90004, usa', 
                '732 huntley dr, 1 west hollywood, ca 90069, usa': '732 huntley dr 1, west hollywood, ca 90069, usa', 
                '2765 1/2 west 12th street, usa': '2765 1/2 west 12th st, los angeles, ca 90006, usa', 
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa', 
                '4710 west maplewood ave los angeles, ca 90004, usa': '4710 west maplewood ave, los angeles, ca 90004, usa', 
                '14333 van nuys blvd. ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa', 
                '1710 n. fuller ave, ca 90046, usa': '1710 n fuller ave, los angeles, ca 90046, usa', 
                '14618 gilmore st, van nuys, 10, ca 91411, usa': '14618 gilmore st 10, van nuys, ca 91411, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia street, los angeles, ca 90015, usa',
                '1019 s catalina ct, unit 508 los angeles, ca 90006, usa': '1019 s catalina ct unit 508, los angeles, ca 90006, usa',
                '1118 valencia street angeles, ca 90015': '1118 valencia street, los angeles, ca 90015'}

# convert everything to lowercase temporarily
cityla_df['city'] = cityla_df['city'].apply(lambda x: x.lower())

# map revised city names
cityla_df['city'] = cityla_df['city'].replace(city_dict)


# TROUBLESOME cities
- 3247 kenneth drive
- san pedr
- 2765 1/2 West 12th street
- HSR19-002358: 6711 S Sepulveda Blvd, Los Angeles, CA 90045, USA mailing address 6890 Arizona Ave. Los Angeles Ca 90045


In [ ]:
# fix misspelled city names

city_dict = {'los a':'los angeles', 
             '#1 north hollywood':'north hollywood', 
             'sunlandtujunga':'sunland-tujunga', 
             'sherma':'sherman oaks', 
             '14333 van nuys blvd':'arleta', 
             '830 e kensington rd los angeles': 'los angeles',
             'los1118 valencia street angeles': 'los angeles', 
             '518 e 21st street los angeles':'los angeles', 
             '707 brooks ave venice':'venice', 
             'apt 103 los angeles': 'los angeles',
             '#b venice': 'venice',
             '1506 n mccollum place unit a los angeles':'los angeles',
             '7641 tujunga ave north hollywood':'north hollywood', 
             'n. sherman oaks':'sherman oaks', 
             '2763 1/2 w 12th st los angeles':'los angeles', 
             'los angeles b':'los angeles',
             'unit c los angeles':'los angeles', 
             '4712 west maplewood ave los angeles':'los angeles', 
             '1 west hollywood':'west hollywood', 
             '2765 1/2 west 1':'los angeles',
             'los1118 valencia street  angeles':'los angeles', 
             '4710 west maplewood ave los angeles':'los angeles', 
             '14333 van nuys blvd.':'arleta', 
             '1710 n. fuller ave': 'los angeles',
             '10': 'van nuys',
             'san pedr': 'san pedro'}


# Checkpoint 3: fix up addresses and city entries

# add ', usa' string back to registered address
#cityla_df['address1'] = cityla_df['address1'] + ', USA'

# split string by delimeter (,)
cityla_df['holder_name_list'] = cityla_df['Permit Holder Name 1 '].str.split(' ')

# select first name
cityla_df.loc[:, 'first_name'] = cityla_df['holder_name_list'].map(lambda x: x[0])

# remove unnecessary characters in first names
cityla_df['first_name'] = cityla_df['first_name'].str.replace('[^a-zA-Z]', '')


cityla_df['city'] = cityla_df['city'].str.strip()


# fix misspelled city names
city_dict = {'los a':'los angeles', 
             '#1 north hollywood':'north hollywood', 
             'sunlandtujunga':'sunland-tujunga', 
             'sherma':'sherman oaks', 
             '14333 van nuys blvd':'arleta', 
             '830 e kensington rd los angeles': 'los angeles',
             'los1118 valencia street angeles': 'los angeles', 
             '518 e 21st street los angeles':'los angeles', 
             '707 brooks ave venice':'venice', 
             'apt 103 los angeles': 'los angeles',
             '#b venice': 'venice',
             '1506 n mccollum place unit a los angeles':'los angeles',
             '7641 tujunga ave north hollywood':'north hollywood', 
             'n. sherman oaks':'sherman oaks', 
             '2763 1/2 w 12th st los angeles':'los angeles', 
             'los angeles b':'los angeles',
             'unit c los angeles':'los angeles', 
             '4712 west maplewood ave los angeles':'los angeles', 
             '1 west hollywood':'west hollywood', 
             '2765 1/2 west 1':'los angeles',
             'los1118 valencia street  angeles':'los angeles', 
             '4710 west maplewood ave los angeles':'los angeles', 
             '14333 van nuys blvd.':'arleta', 
             '1710 n. fuller ave': 'los angeles',
             '10': 'van nuys'}


# fix these addresses manually
address_dict = {'10834 hortense st, #1 north hollywood, ca 91602, usa':'10834 hortense st, north hollywood, ca 91602, usa',
                
                'w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91411, usa',
                '13260 w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91423, usa',
                
                '14333 van nuys blvd. ca 91331, usa':'14333 van nuys blvd, arleta, ca 91331, usa',
                '830 e kensington rd los angeles, ca 90026, usa':'830 e kensington rd, los angeles, ca 90026, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa',
                '518 e 21st street los angeles, ca 90011, usa':'518 e 21st st, los angeles, ca 90011, usa',
                '707 brooks ave venice, ca 90291, usa':'707 brooks ave, venice, ca 90291, usa',
                '1255 federal ave, apt 103 los angeles, ca 90025, usa':'1255 federal ave, los angeles, ca 90025, usa',
                '344 indiana ave, #b venice, ca 90291, usa':'344 indiana ave, venice, ca 90291, usa',
                '1506 n mccollum place unit a los angeles, ca 90026, usa':'1506 n mccollum place, los angeles, ca 90026, usa',
                '7641 tujunga ave north hollywood, ca 91605, usa': '7641 tujunga ave, north hollywood, ca 91605, usa',
                '6828 van nuys blvd, n. sherman oaks, ca 91405, usa': '6828 van nuys blvd, sherman oaks, ca 91405, usa', 
                '2763 1/2 w 12th st los angeles, ca 90006, usa': '2763 1/2 w 12th st, los angeles, ca 90006, usa', 
                '1474 ridge way, los angeles b, ca 90026, usa': '1474 ridge way, los angeles, ca 90026, usa', 
                '6650 leland way, unit c los angeles, ca 90028, usa': '6650 leland way, los angeles, ca 90028, usa', 
                '4712 west maplewood ave los angeles, ca 90004, usa': '4712 west maplewood ave, los angeles, ca 90004, usa', 
                '732 huntley dr, 1 west hollywood, ca 90069, usa': '732 huntley dr, west hollywood, ca 90069, usa', 
                '2765 1/2 west 12th street, usa': '2765 1/2 west 12th st, los angeles, ca 90006, usa', 
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa', 
                '4710 west maplewood ave los angeles, ca 90004, usa': '4710 west maplewood ave, los angeles, ca 90004, usa', 
                '14333 van nuys blvd. ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa', 
                '1710 n. fuller ave, ca 90046, usa': '1710 n fuller ave, los angeles, ca 90046, usa', 
                '14618 gilmore st, van nuys, 10, ca 91411, usa': '14618 gilmore st, van nuys, ca 91411, usa'}




# uni, ca, zephyr cove??, 10 san fernando

# map revised addresses
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].replace(address_dict)

# map revised city names
cityla_df['city'] = cityla_df['city'].replace(city_dict)


cityla_df.head()

cityla_df['city'].value_counts()

# remove rows with empty values
#cityla_df.loc[cityla_df['city'] == '']


# why is there nevada listings registered?
cityla_df.loc[cityla_df['city'] == 'zephyr cove']

# replace values with Nan
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].replace('', np.nan)

# drop rows with nan in registered address
cityla_df = cityla_df.dropna(subset = ['Registered Adddress'])
          

cityla_df.head()

cityla_df.shape

#os.chdir(BASE_DIR + 'data/intermediate/')
#cityla_df.to_csv('cityla_merged_v3.csv', index = False)


# SCRATCHPAD: old code that Karina was using to clean up the registrant dataset

[Github coordinates tutorial](https://github.com/code-and-dogs/seleniumBsFolium/blob/master/getGeo.py)

[Youtube video link](https://www.youtube.com/watch?v=tKy-IHAxt4s)


# Merge city of LA registerant datasets

os.chdir(BASE_DIR + 'data/raw/cityla/')

xls = pd.ExcelFile('HSO Registrants over time2.xlsx')
#df1 = pd.read_excel(xls, 'Sheet1')
#df2 = pd.read_excel(xls, 'Sheet2')

#cityla_df = pd.read_excel('HSO Registrants over time2.xlsx', sheet_name = None)


cityla_df1 = pd.read_excel(xls, '12-12-19')
cityla_df2 = pd.read_excel(xls, '1-16-20')
cityla_df3 = pd.read_excel(xls, '2-7-20')
cityla_df4 = pd.read_excel(xls, '3-13-20')
cityla_df5 = pd.read_excel(xls, '4-14-20')
cityla_df6 = pd.read_excel(xls, '5-27-20')
cityla_df7 = pd.read_excel(xls, '7-9-20')


cityla_df = pd.merge(cityla_df1, cityla_df2, how = 'outer')
cityla_df = pd.merge(cityla_df, cityla_df3, how = 'outer')
cityla_df = pd.merge(cityla_df, cityla_df4, how = 'outer')
cityla_df = pd.merge(cityla_df, cityla_df5, how = 'outer')
cityla_df = pd.merge(cityla_df, cityla_df6, how = 'outer')
cityla_df = pd.merge(cityla_df, cityla_df7, how = 'outer')

print(cityla_df1.shape)
print(cityla_df2.shape)
print(cityla_df3.shape)
print(cityla_df4.shape)
print(cityla_df5.shape)
print(cityla_df6.shape)
print(cityla_df7.shape)

cityla_df.shape

cityla_df.head()

#os.chdir(BASE_DIR + 'data/intermediate/')
#cityla_df.to_csv('cityla_merged_v1.csv', index = False)

# Checkpoint 1: Merge LA city dataset with Airbnb registered listings

os.chdir(BASE_DIR + 'data/intermediate/')
cityla_df = pd.read_csv('cityla_merged_v1.csv')

# convert everything to lowercase
cityla_df = cityla_df.apply(lambda x: x.astype(str).str.lower())


# check for NAs in the Platforms Reported at Application column
cityla_df['Platforms Reported at Application'].isnull().values.any()


# check for NAs in the Registered Adddress column
cityla_df['Registered Adddress'].isnull().values.any()



# only keep registerants that match this list exactly
remove_list = ['-', 'a']
mask1 = cityla_df[cityla_df['Platforms Reported at Application'].isin(remove_list)]


mask1['Platforms Reported at Application'].value_counts()

# now keep if any strings CONTAIN airbnb
keep = ['airbnb']
mask2 = cityla_df[cityla_df['Platforms Reported at Application'].str.contains('|'.join(keep))]


# now merge them together
cityla_df = pd.concat([mask1, mask2])

cityla_df.shape

cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].str.replace('\t','')
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].str.replace('-','')


# remove unnecessary characters
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].apply(lambda x: x.split(', usa')[0])


cityla_df.head(n = 3)

cityla_df.columns

# some permit holders have more than one registeration numbers
grouped_cityla_df = cityla_df.groupby(['Registered Adddress', 'Permit Holder Name 1 '])['Registration Number'].nunique()


cityla_df['Registered Unit Number'] = cityla_df['Registered Unit Number'].replace('-', np.nan)

cityla_df['Platforms Reported at Application'] = cityla_df['Platforms Reported at Application'].replace('-', np.nan)
cityla_df['Platforms Reported at Application'] = cityla_df['Platforms Reported at Application'].replace('a', np.nan)


cityla_df['Registered Unit Number'].val

# create new csv
#os.chdir(BASE_DIR + 'data/intermediate/')
#cityla_df.to_csv('cityla_merged_v2.csv', index = False)


# Checkpoint 2: fix up addresses and city entries

os.chdir(BASE_DIR + 'data/intermediate/')
cityla_df = pd.read_csv('cityla_merged_v2.csv')


cityla_df.head()


# remove white space at the beginning and end of a string
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].str.strip()
cityla_df['Permit Holder Name 1 '] = cityla_df['Permit Holder Name 1 '].str.strip()


# ADD HERE

remove_address_char = [', united states', ' usa', ', us', ' usa']
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].str.replace('|'.join(remove_address_char), '')





# remove last 10 characters
cityla_df['city'] = cityla_df['Registered Adddress'].map(lambda x: str(x)[:-10])

# split string by delimeter (,)
cityla_df['city'] = cityla_df['city'].str.split(',')

# create city and street address columns
cityla_df.loc[:, 'city'] = cityla_df['city'].map(lambda x: x[-1])


# add ', usa' string back to registered address
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'] + ', usa'


cityla_df.head()

# split string by delimeter (,)
cityla_df['holder_name_list'] = cityla_df['Permit Holder Name 1 '].str.split(' ')



# select first name
cityla_df.loc[:, 'first_name'] = cityla_df['holder_name_list'].map(lambda x: x[0])


# remove unnecessary characters in first names
cityla_df['first_name'] = cityla_df['first_name'].str.replace('[^a-zA-Z]', '')


cityla_df.head(n = 3)

cityla_df['city'] = cityla_df['city'].str.strip()


# fix misspelled city names
city_dict = {'los a':'los angeles', 
             '#1 north hollywood':'north hollywood', 
             'sunlandtujunga':'sunland-tujunga', 
             'sherma':'sherman oaks', 
             '14333 van nuys blvd':'arleta', 
             '830 e kensington rd los angeles': 'los angeles',
             'los1118 valencia street angeles': 'los angeles', 
             '518 e 21st street los angeles':'los angeles', 
             '707 brooks ave venice':'venice', 
             'apt 103 los angeles': 'los angeles',
             '#b venice': 'venice',
             '1506 n mccollum place unit a los angeles':'los angeles',
             '7641 tujunga ave north hollywood':'north hollywood', 
             'n. sherman oaks':'sherman oaks', 
             '2763 1/2 w 12th st los angeles':'los angeles', 
             'los angeles b':'los angeles',
             'unit c los angeles':'los angeles', 
             '4712 west maplewood ave los angeles':'los angeles', 
             '1 west hollywood':'west hollywood', 
             '2765 1/2 west 1':'los angeles',
             'los1118 valencia street  angeles':'los angeles', 
             '4710 west maplewood ave los angeles':'los angeles', 
             '14333 van nuys blvd.':'arleta', 
             '1710 n. fuller ave': 'los angeles',
             '10': 'van nuys'}


# fix these addresses manually
address_dict = {'10834 hortense st, #1 north hollywood, ca 91602, usa':'10834 hortense st, north hollywood, ca 91602, usa',
                
                'w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91411, usa',
                '13260 w magnolia blvd, sherman oaks, ca, usa':'13260 w magnolia blvd, sherman oaks, ca 91423, usa',
                
                '14333 van nuys blvd. ca 91331, usa':'14333 van nuys blvd, arleta, ca 91331, usa',
                '830 e kensington rd los angeles, ca 90026, usa':'830 e kensington rd, los angeles, ca 90026, usa',
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa',
                '518 e 21st street los angeles, ca 90011, usa':'518 e 21st st, los angeles, ca 90011, usa',
                '707 brooks ave venice, ca 90291, usa':'707 brooks ave, venice, ca 90291, usa',
                '1255 federal ave, apt 103 los angeles, ca 90025, usa':'1255 federal ave, los angeles, ca 90025, usa',
                '344 indiana ave, #b venice, ca 90291, usa':'344 indiana ave, venice, ca 90291, usa',
                '1506 n mccollum place unit a los angeles, ca 90026, usa':'1506 n mccollum place, los angeles, ca 90026, usa',
                '7641 tujunga ave north hollywood, ca 91605, usa': '7641 tujunga ave, north hollywood, ca 91605, usa',
                '6828 van nuys blvd, n. sherman oaks, ca 91405, usa': '6828 van nuys blvd, sherman oaks, ca 91405, usa', 
                '2763 1/2 w 12th st los angeles, ca 90006, usa': '2763 1/2 w 12th st, los angeles, ca 90006, usa', 
                '1474 ridge way, los angeles b, ca 90026, usa': '1474 ridge way, los angeles, ca 90026, usa', 
                '6650 leland way, unit c los angeles, ca 90028, usa': '6650 leland way, los angeles, ca 90028, usa', 
                '4712 west maplewood ave los angeles, ca 90004, usa': '4712 west maplewood ave, los angeles, ca 90004, usa', 
                '732 huntley dr, 1 west hollywood, ca 90069, usa': '732 huntley dr, west hollywood, ca 90069, usa', 
                '2765 1/2 west 12th street, usa': '2765 1/2 west 12th st, los angeles, ca 90006, usa', 
                'los1118 valencia street angeles, ca 90015, usa': '1118 valencia st, los angeles, ca 90015, usa', 
                '4710 west maplewood ave los angeles, ca 90004, usa': '4710 west maplewood ave, los angeles, ca 90004, usa', 
                '14333 van nuys blvd. ca 91331, usa': '14333 van nuys blvd, arleta, ca 91331, usa', 
                '1710 n. fuller ave, ca 90046, usa': '1710 n fuller ave, los angeles, ca 90046, usa', 
                '14618 gilmore st, van nuys, 10, ca 91411, usa': '14618 gilmore st, van nuys, ca 91411, usa'}




# uni, ca, zephyr cove??, 10 san fernando

# map revised addresses
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].replace(address_dict)

# map revised city names
cityla_df['city'] = cityla_df['city'].replace(city_dict)


cityla_df.head()

cityla_df['city'].value_counts()

# remove rows with empty values
#cityla_df.loc[cityla_df['city'] == '']


# why is there nevada listings registered?
cityla_df.loc[cityla_df['city'] == 'zephyr cove']

# replace values with Nan
cityla_df['Registered Adddress'] = cityla_df['Registered Adddress'].replace('', np.nan)

# drop rows with nan in registered address
cityla_df = cityla_df.dropna(subset = ['Registered Adddress'])
          

cityla_df.head()

cityla_df.shape

#os.chdir(BASE_DIR + 'data/intermediate/')
#cityla_df.to_csv('cityla_merged_v3.csv', index = False)


# Checkpoint 3: Begin creating coordinates

[Github coordinates tutorial](https://github.com/code-and-dogs/seleniumBsFolium/blob/master/getGeo.py)

[Youtube video link](https://www.youtube.com/watch?v=tKy-IHAxt4s)
